# CREDIT DEFAULT PROTOTYPE MODEL

Author: Adrian Bergem <br>
Date: 31/10/2018

# Summary
This notebook is a prototype for a credit default model for client xyz. <br>

The are 3 <b>datasets</b> available, namely borrower, loan and payments. All three datasets have 887 379 rows and 14, 18 and 41 columns respectively. As the payment data seems to be dated recently, I argue it cannot be used in a model where one is trying to predict credit default straight after receiving their loan approval. I also omit data from loans that were originated prior to 2010 to avoid market dynamics from the financial crisis. 

Loan status is used to construct the <b>target variables</b>. I find it reasonable to only include the loans that are terminated – i.e. the ones with a status as either 'Charged Off' or 'Fully Paid' - in order to compare apples to apples. The stratified train-test split shows a class imbalance where positives (defaults) make up 18% of the train data.  I have chosen not to oversample/undersample the data due its shortcomings (loss of training data or reusage of data causing bias) compared to the relatively low imbalance.

The <b>EDA</b> is quite extensive as I go through all borrower and loan features in detail, evaluating their usefulness. The features that seem to be most important (verified by the model’s features importance) are address state (US state provided by the of the borrower’s address), sub-grades (granular grade assigned by the client) and annual income of the borrower.

The <b>data pre-processing</b> consists of cleaning some of the features, dropping the ones with (almost) no variance, dropping those that correlate >99% with other features, dropping the ones with >90% NaNs, and those that require too much feature engineering to make use of. Some string features such as 'Description' and 'Employment Title' could be worth looking more into with more feature engineering.

As <b>scoring metric</b>, I'm using roc-auc as in this case one might argue that predicting both classes is important, and hence focusing on both TPR and FPR. I would think a company with a business model as the intermediate between borrowers and lenders would benefit from a decent FPR to not forego too much business opportunities.

The <b>modeling</b> is done by first spot-checking 3 different models, namely logistic regression with the SGD algorithm, sklearn's Random Forest and LightGBM. Showing a consistenly higher auc (above 0.70), I progress with the LightGBM to find the best hyperparameters using Bayesian Optimization (with the Hypopt library). Due to its requirement for computational power, I use a sample of the training data to perform the kfold cv iterations to find optimal hyperparameters. Using these optimal hyperparameters on a kfold cv on the full training set actually shows a slightly lower auc score than with the spot-checked parameters (but still above 0.70). This is likely due to the sampling of the training data. On the bright side, the optimization shows a positive trend in score with respect to the number of iterations, and would likely lead to a higher score when using the full training set for optimization. When developing the model for production, I would thus suggest to use bayesian optimization for the hyperparameters, but training it on the full training set.

The <b>results on the test set</b> are good, with a roc-auc of ~0.71 – showing no signs of overfitting. The model makes it easy to experiment with different thresholds for class prediction, and hence work with the TPR/FPR trade-off. I think this should be done along with the client to agree on which metrics are most important. 

To assess the model's <b>business value</b>, I compare it to a model that might be similar to the one being used today, i.e. a model based on sub-grades. The subgrade model is built by allocating a probability of default linearly across the sub-grades (from 0% on A1 to 50% on G5). By tweeking the threshold of the subgrade model to match the ML model's number of TPs, one can compare the performance by looking at the confusion matrices. The ML model clearly outperforms the subgrade model with 201 less FNs and 1307 less FPs, clearly demonstrating the ML model’s added business value. 

The <b>limitation</b> of the model, as briefly mentioned, is that I have not treated the classes imbalance. Although the imbalance isn't as bad as many other applications, one could possibly obtain improvements by using sklearn's RandomUnderSampler() and LightGBM's scale_pos_weight parameter. I am though not too comfortable with those techniques, which is the reason why I haven't tried them out here. For the next stage, when working with a model to be deployed into production, I would suggest to try out these techniques.


# Contents

1. Libs imports and settings <br>
<br>
2. Import data and quick inspection <br>
<br>
3. Setting up the problem <br>
    3.1 Merge dataset <br>
    3.2 Overview of credit default <br>
    3.3 Set up the target variable <br>
    3.4 Preliminary data cleaning: remove low-value features <br>
    3.5 Omit old data <br>
    3.6 Split train and test data <br>
<br>
4. EDA <br>
    4.1 Functions <br>
    4.2 Data on borrower/demographics <br>
    4.3 Loan data <br>
    4.4 Feature engineering <br>
<br>
5. Data pre-processing <br>
<br>
6. Modeling <br>
    6.1 Spot-check 3 different algorithms using grid search cross-validation <br>
    6.2 Bayesian Hyperparameter Optimization <br>
    6.3 Results of best model on full train data <br>
    6.4 Feature importances with best model <br>
<br>
7. Results on test data <br>
    7.1 Pre-process test data <br>
    7.2 Confusion matrix <br>
    7.3 ROC curve and ROC-AUC score <br>
    7.4 Comparison with sub-grade model

# 1. Libs imports and settings

<b>Imports:</b>

In [3]:
# File system management
import os

import csv
from timeit import default_timer as timer

# Warnings
import warnings

# numpy and pandas for data wrangling
import numpy as np
import pandas as pd
import re

# datetime libs
from datetime import datetime
from dateutil.parser import parse

# matplotlib, seaborn and wordcloud for plotting
import matplotlib.pyplot as plt
import seaborn as sns

# itertools for efficient looping
import itertools

# Wordcloud for exploring string features
#from wordcloud import WordCloud

# scikit-learn for modeling
from sklearn.preprocessing import Imputer, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_predict
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve, roc_auc_score, confusion_matrix, classification_report
from sklearn.externals import joblib

# LightGBM for gradient boosting
import lightgbm as lgb

# Hyperopt for bayesian hyperparameter optimization
#from hyperopt import hp
#from hyperopt.pyll.stochastic import sample
#from hyperopt import tpe
#from hyperopt import Trials
#from hyperopt import fmin
#from hyperopt import STATUS_OK
#import ast

<b>Settings:</b>

In [6]:
# Print settings
from pprint import pprint
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

# Set working directory
os.chdir(r'C:\Users\adrian.bergem\Google Drive\Data science\Projects\AI Credit Default')

# Ignore warnings
warnings.filterwarnings('ignore')

# Global modeling settings
seed = 42

# Global plotting settings
%matplotlib inline
sns.set(style='whitegrid')

# 2. Import data and quick inspection

Import the 3 various csv files (the payment csv was created from the .rda R file using the following R code: <br>
payments <- load("\\Loan Data\\Loan Payment Information.Rda")
write.csv(Payment.df, file = "Loan_payment_information.csv")

In [8]:
borrower = pd.read_csv(r'data\raw\Borrower Information.csv')

In [9]:
loan = pd.read_csv(r'data\raw\Loan Classification Information.csv', low_memory=False)

In [ ]:
payment = pd.read_csv('Loan Data\\Loan_payment_information.csv')

Next, I will inspect the data to get an overview

In [ ]:
print('Borrower dataset shape:', borrower.shape)
print('Loan dataset shape:', loan.shape)
print('Payment dataset shape:', payment.shape)

In [ ]:
borrower.head()

In [ ]:
borrower.info()

In [ ]:
loan.head()

In [ ]:
loan.info()

In [ ]:
payment.head()

In [ ]:
payment.info()

# 3. Setting up the problem

One of the most important question is what the time stamp of the data we have at hand is. If we are to train a model on historical data, we need to be careful not to include forward-looking data.

One needs to make the assumption that loan and demographics data is from the time of the loan origination. The payments data seems to be recent data, and can thus not be used.

## 3.1 Merge dataset

In [ ]:
# Count number of unique customers per loan
loan['member_id'].nunique()

There seems to be only 1 loan per borrower, hence there is a 1:1 relationship between the borrower and the loan dataset. We can thus merge the 2 datasets on the member_id key (primary key in borrower and foreign key in loan)

In [10]:
df = pd.merge(borrower, loan, on='member_id')

In [11]:
# Double-check that everything went as planned
df.shape

(887379, 31)

## 3.2 Overview of credit default

### 3.2.1 Overview of loan statuses

Let's get an overview of the credit default by inspecting the loan statuses

In [ ]:
df['loan_status'].value_counts().plot.barh()
plt.show()

I will simplify the visual assessment by treating late payments equally. For modeling, I will only include 'Fully Paid' and 'Charged Off', to be able to compare apples with apples (both terminated). For now, I will leave in 'Current' and 'Late' for getting an overview of the business.

In [ ]:
df.replace('Late (16-30 days)', 'Late', inplace=True)
df.replace('Late (31-120 days)', 'Late', inplace=True)

In [ ]:
df = df.loc[df['loan_status'].isin(['Late', 'Fully Paid', 'Charged Off', 'Current'])]

In [ ]:
df['issue_d'] = pd.to_datetime(df['issue_d'])

In [ ]:
by_issue = df.groupby(['issue_d', 'loan_status'])['id'].count().unstack()

In [ ]:
# Resample on quarterly basis
by_issue_resampled = by_issue.resample('Q').sum()

In [ ]:
# Plot loan statuses over issue time
fig, ax = plt.subplots(1,2, figsize=(12,6))
by_issue_resampled.drop('Current', axis=1).plot(ax=ax[0])
ax[0].set_title('Number of loans by status')
ax[0].set_xlabel('Issue date')
ax[0].set_ylabel('Loans')
by_issue_resampled.plot(ax=ax[1])
ax[1].set_title('Number of loans by status incl. current')
ax[1].set_xlabel('Issue date')
ax[1].set_ylabel('Loans')

### 3.2.2 Default rate of terminated loans

Let's have a look at the historical default rate.

In [12]:
# Limit the dataset further into only terminated ones
df = df[df['loan_status'].isin(['Charged Off', 'Fully Paid'])]

In [ ]:
df['loan_status'].value_counts()/len(df)

In [ ]:
# Plot the percentage distribution of loan status
fig, ax = plt.subplots(figsize=(6,4))
df.groupby('loan_status')['loan_status'].count().apply(
    lambda x: int(x/len(df_tmp)*100)).plot.bar(ax=ax, rot=0, title='Percentage of borrowers')
ax.set_xlabel('')
ax.set_ylabel('Percentage')

### 3.2.2 Financial impact: losses

To get a sense of the financial implication of the defaults, I will look at historical losses. For this I will use the loan and payment datasets. 

In [ ]:
# Create a temporary dataframe with payment data to calculate losses
df_tmp = pd.merge(payment, loan, on='id')

In [ ]:
payment_defaults = df_tmp[df_tmp['loan_status'] == 'Charged Off']

In [ ]:
recovery_rate = payment_defaults['recoveries'] / (payment_defaults['loan_amnt']-payment_defaults['total_rec_prncp'])

In [ ]:
recovery_rate.describe()

There seems to be something fishy about the recoveries data, so in the calcualtion of losses, we will just use the median recovery rate

In [ ]:
losses = payment_defaults['loan_amnt']-payment_defaults['total_rec_prncp'] * (1-recovery_rate.median())

In [ ]:
# Plot distribution of historical losses
fig, ax = plt.subplots(figsize=(8,4))
sns.distplot(losses, ax=ax)
ax.set_title('Distribution of historical losses (estimated)')

The sum of historical losses has amounted to around 0.5bn 

In [ ]:
print(losses.sum())

## 3.3 Set up the target variable

I will use the loan status as the target variable for the supervised learning setup.

With only terminated loans, we are down to 252k rows (full dataset with ~800k), but it is more than enough for getting descent ML models

In [ ]:
df.shape

In [13]:
# Transform the loan status variable to 1s and 0s
df['target'] = df['loan_status'].apply(lambda x: 1 if x == 'Charged Off' else 0)

In [14]:
# Drop the loan status feature as we will no longer need it
df.drop(labels='loan_status', axis=1, inplace=True)

In [ ]:
df.info()

## 3.4 Preliminary data cleaning: remove low-value features

We have a handful of features in the data that will not be of any use for modeling. We remove the id columns, and the features with a lot of NaNs

In [15]:
# Remove id columns
df.drop(labels=['Unnamed: 0_x', 'Unnamed: 0_y', 'member_id', 'id'], axis=1, inplace=True)

In [16]:
# Create dataframe for NaNs
nans = pd.DataFrame(index=df.columns, columns=['percentage NaNs'])

In [17]:
# Transform NaN counts to percentages
for feature in df.columns:
    nans.loc[feature] = df[feature].isnull().sum()/(len(df))

In [18]:
nans.sort_values(by='percentage NaNs', ascending=False)

,percentage NaNs
annual_inc_joint,0.999996
desc,0.650873
emp_title,0.0550696
emp_length,0.0391231
title,5.13893e-05
pymnt_plan,0
installment,0
funded_amnt_inv,0
funded_amnt,0
int_rate,0


In [19]:
# Remove features with more than 90% NaNs
cols_to_remove = nans[nans['percentage NaNs'] > 0.90].index.tolist()
df.drop(labels=cols_to_remove, axis=1, inplace=True)

## 3.5 Omit old data

One might argue that due to the abnormal credit markets during the financial crisis, the dynamics 2007-2010 were somewhat different. From the historical loan issue data, we also saw very low figures during the years prior to 2010, which might indicate the company to be in its infancy. Based on this, I choose to exclude datapoints prior to 2010.

In [20]:
df = df[df['issue_d'] > '01-01-2010']

In [ ]:
df.shape

## 3.6 Split train and test data

I'm using stratified splitting between train and test data with respect to the target variable. This is to keep the imbalance equal between the two.

In [21]:
# Split data into target (y) and features (X)
X = df.drop(labels='target', axis=1)
y = df['target']

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed, stratify=y)

Check that the target imbalance is equivalent

In [ ]:
y_train.value_counts().divide(len(y_train))

In [ ]:
y_test.value_counts().divide(len(y_test))

Reset indexes in train and test datasets

In [26]:
X_train.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)
X_test.reset_index(drop=True, inplace=True)
y_test.reset_index(drop=True, inplace=True)

In [ ]:
print('X_train shape is', X_train.shape)
print('y_train shape is', y_train.shape)

I will use a merged dataset for data visualization on the training data (to be able to visualize each feature wrt target)

In [27]:
# Recombine features and target in train data
train = pd.concat([y_train, X_train], axis=1).reset_index(drop=True)

In [ ]:
train.info()

# 4. EDA

I will go through each feature one by one to assess its usefulness and get to know the training data

## 4.1 Functions

In [ ]:
def remove_outliers(df, feature, threshold=2):
    """
    Computes outliers based on standard deviation from the mean (set to 2 as default)
    Returns the input dataframe without outliers
    """
    mean = df[feature].mean()
    std = df[feature].std()
    upper = mean + 2*std
    lower = mean - 2*std
    outliers = df[(df[feature] > upper) | (df[feature] < lower)][feature]
    return df[~df[feature].isin(outliers)]

In [ ]:
def plot_feature(df, feature, name, dtype, keep_outliers=True, size='normal'):
    """
    Plots features where one gets to choose between 3 different options in the dtype parameter
        If numerical: 
            left: distplot with kde 
            right: violinplot wrt target
        If categorical: 
            left: countplot
            right: barplot with central tendency wrt target
        If other:
            left: Distribution of defaulters wrt date feature
            right: Distribution of non-defaulters wrt date feature
    """
    
    # Create temporary dataframe that we can manipulate how we want
    df_tmp = df.copy()
    df_tmp = df_tmp[df_tmp[feature].notnull()]
    df_tmp['target_desc'] = df_tmp['target'].map({0: 'Non-default', 1: 'Default'})
    
    #Remove outliers to better visualize the feature
    if keep_outliers==False:
        df_tmp = remove_outliers(df_tmp, feature)
    
    if size == 'normal':
        fig, ax = plt.subplots(1, 2, figsize=(14,4))
    elif size == 'big':
        fig, ax = plt.subplots(1, 2, figsize=(14,8))
    
    fig.suptitle(name)
    
    # Numerical features
    if dtype == 'num':
        
        # left plot
        sns.distplot(df_tmp[feature], kde=True, ax=ax[0])
        ax[0].set_title('Distribution')
        ax[0].set_xlabel(name)
        ax[0].set_ylabel('Count')
        
        # right plot
        sns.violinplot(x=feature, y='target_desc', data=df_tmp, ax=ax[1])
        ax[1].set_title('Distribution of defaulters vs. non-defaulters')
        ax[1].set_xlabel(name)
        ax[1].set_ylabel('')
    
    elif dtype == 'cat':
              
        # left plot
        sns.countplot(y=df_tmp[feature], order=sorted(df[feature].unique()), ax=ax[0])
        ax[0].set_title('Count of loans')
        ax[0].set_ylabel(name)
        ax[0].set_xlabel('Count') 
            
        # Right axis object
        sns.barplot(y=feature, x='target', order=sorted(df[feature].unique()), data=df_tmp, ax=ax[1])
        ax[1].set_title('Feature central tendency wrt default')
        ax[1].set_ylabel('')
        ax[1].set_xlabel('Central tendency of default')
        
    elif dtype == 'other':
        
        # left plot
        df_tmp[df_tmp['target']==1][feature].hist(ax=ax[0], bins=20, color='darkred')
        ax[0].set_title('Distribution for defaulters')
        ax[0].set_xlabel(name)
        ax[0].set_ylabel('Count')
        
        # right plot
        df_tmp[df_tmp['target']==0][feature].hist(ax=ax[1], bins=20, color='darkblue')
        ax[1].set_title('Distribution for non-defaulters')
        ax[1].set_xlabel(name)
        ax[1].set_ylabel('Count')

## 4.2 Data on borrower/demographics

### 4.2.1 State address

In [ ]:
train['addr_state'].dtype

In [ ]:
plot_feature(train, feature='addr_state', name='US state of borrower address', dtype='cat', size='big')

There seems to be huge variances across states on default!

### 4.2.2 Annual income

In [ ]:
train['annual_inc'].dtype

In [ ]:
plot_feature(train, feature='annual_inc', name='Borrower Annual Income', dtype='num', keep_outliers=False)

Non-defaulters seem to have higher income, which is hardly surprising

### 4.2.3 Delinquency incidences

In [ ]:
train['delinq_2yrs'].dtype

In [ ]:
plot_feature(train, feature='delinq_2yrs', name='Delinquency Incidences', dtype='other', keep_outliers=True)

There seems to be more deliquencies the last 2 years prior to the loan application for defaulters

### 4.2.4 Earliest report credit line

In [ ]:
train['earliest_cr_line'].dtype

In [ ]:
# Create temporary variable of the feature in datetime type
train['tmp'] = pd.to_datetime(train['earliest_cr_line'])

In [ ]:
plot_feature(train, feature='tmp', name='Earliest report credit line', dtype='other', keep_outliers=True)

In [ ]:
# Drop temporary feature
train.drop(labels='tmp', axis=1, inplace=True)

There is some variance in the data wrt target that could be useful for modeling

### 4.2.5 Employment length

Employment length is stored in text as categories

In [ ]:
train['emp_length'].dtype

In [ ]:
train['tmp'].unique()

In [ ]:
plot_feature(train, feature='emp_length', name='Employment length', dtype='cat', keep_outliers=True)

There seems to be some relationship between employment length and default. This variable will be processed to numericals prior to modeling though, to capture the continuous nature of the variable

### 4.2.6 Employment title

Employment title is a string variable that needs some extensive feature engineering to be useful. In interest of time, that will be left to a later stage.

In [ ]:
train['emp_title'].dtype

In [ ]:
train['emp_title'].value_counts()

In [ ]:
# Group the train dataset by feature and aggregate on count and mean of target
defaults_by_title = train.groupby('emp_title')['target'].agg(['count', 'mean'])

In [ ]:
defaults_by_title = defaults_by_title[defaults_by_title['count']>50]

In [ ]:
# Show top 5 titles associated with the highest default rates
defaults_by_title.sort_values(by='mean', ascending=False)[:5]

Typical blue-collar titles have the highest default mean. Interesting, but extensive feature engineering would be needed to make this variable more valuable in a model.

### 4.2.7 Home ownership

Home ownership is a categorical variable. I merge the values 'none' and 'any' into the 'other' value to make it less fragmented

In [ ]:
train['home_ownership'].dtype

In [ ]:
train['home_ownership'].value_counts()

In [ ]:
train['tmp'] = train['home_ownership'].replace(to_replace=['NONE', 'ANY'], value='OTHER')

In [ ]:
plot_feature(train, feature='tmp', name='Home ownership', dtype='cat', keep_outliers=True)

In [ ]:
train.drop(labels='tmp', axis=1, inplace=True)

Borrowers with a mortgage seem to have lower default rates, whereas borrowers renting show the opposite

### 4.2.8 Open credit lines

In [ ]:
train['open_acc'].dtype

In [ ]:
plot_feature(train, feature='open_acc', name='Open credit lines', dtype='num', keep_outliers=True)

Hard to see any noteworthy variance in open credit lines with respect to default. I will keep it as it might add value in combination with other variables.

### 4.2.9 Number of historic credit lines

In [ ]:
train['total_acc'].dtype

In [ ]:
plot_feature(train, feature='total_acc', name='Historic credit lines', dtype='num', keep_outliers=True)

Same story as with open credit lines

### 4.2.10 Number of derogatory public records

In [ ]:
train['pub_rec'].dtype

In [ ]:
train['pub_rec'].value_counts()

In [ ]:
plot_feature(train, feature='pub_rec', name='Number of derogatory public records', dtype='other', keep_outliers=True)

No apparent relationship, but we will keep it either way

### 4.2.11 Zip code

Zip code will include the same data as address, but possibly on a more granular level. However, we only have the first 3 digits in the zip code. I decide to drop this and rather use address state.

In [ ]:
train['zip_code'].dtype

In [ ]:
train['zip_code'].sample()

In [ ]:
train['zip_code'].nunique()

## 4.3 Loan data

### 4.3.1 Application type

No variance in this feature, hence we drop it

In [ ]:
train['application_type'].dtype

In [ ]:
train['application_type'].value_counts()

### 4.3.2 Description

I will use the wordcloud to look into this feature, but more extensive string manipulation and feature engineering would be needed to make use of this data. This is something one could explore at a later stage

In [ ]:
train['desc'].dtype

In [ ]:
wc = train['desc'][:100].dropna().apply(lambda x: x.strip()).copy()

In [ ]:
for i, desc in enumerate(wc):
    print(desc[29:]+'\n')

In [ ]:
def create_wordcloud(df, feature):
    wc = df[feature][:10000].dropna().apply(lambda x: x.strip()).copy()
    for i, desc in enumerate(wc):
        if desc[0:8] == 'Borrower':
            wc[i] = desc[29:]
    return wc

In [ ]:
wc = create_wordcloud(train, 'desc')

In [ ]:
wc.sample(10)

In [ ]:
wc = WordCloud().generate(' '.join(wc))

In [ ]:
fig = plt.figure(figsize=(10,8))
plt.imshow(wc)
plt.axis('off')
plt.show()

### 4.3.3 Total amount funded

This data is already included in the 'Loan amount' feature, as shown in the correlation below. It will thus we dropped prior to modeling.

In [ ]:
train['funded_amnt'].dtype

In [ ]:
train['funded_amnt'].corr(train['loan_amnt'])

### 4.3.4 Amount committed by investors

This data is already included in the 'Total amount funded' feature, as shown in the correlation below. It will thus we dropped prior to modeling.

In [ ]:
train['funded_amnt_inv'].dtype

In [ ]:
train['funded_amnt_inv'].corr(train['loan_amnt'])

### 4.3.5 Loan amount

In [ ]:
train['loan_amnt'].dtype

In [ ]:
plot_feature(train, feature='loan_amnt', name='Loan amount', dtype='num', keep_outliers=True)

Defaulters seem to have higher amounts on their loans

### 4.3.6 Loan grade

This data is included, but on a more granular level in the 'Loan sub-grade' feature, thus we I use that instead

In [ ]:
train['grade'].dtype

In [ ]:
train['grade'].value_counts()

### 4.3.7 Loan sub-grade

In [ ]:
train['sub_grade'].dtype

In [ ]:
plot_feature(train, feature='sub_grade', name='Loan sub-grade', dtype='cat', size='big', keep_outliers=True)

There is clearly a significant relationship here, so the grading system seems to be working to some degree.

### 4.3.8 Installment

In [ ]:
train['installment'].dtype

In [ ]:
plot_feature(train, feature='installment', name='Installment', dtype='num', keep_outliers=True)

Defaulters seem to have slightly higher installments

### 4.3.9 Interest rate

In [ ]:
train['int_rate'].dtype

In [ ]:
plot_feature(train, feature='int_rate', name='Interest rate', dtype='num', keep_outliers=True)

As risky borrowers are charged higher interest rates, the relationship between the two is hardly surprising

### 4.3.10 Purpose

In [ ]:
train['purpose'].dtype

In [ ]:
plot_feature(train, feature='purpose', name='Purpose', dtype='cat', size='big', keep_outliers=True)

There is definitely some interesting relationships here.

### 4.3.11 Payment plan

This feature has almost no variance, hence we will drop it prior to modeling

In [ ]:
train['pymnt_plan'].dtype

In [ ]:
train['pymnt_plan'].value_counts()

### 4.3.12 Term

In [ ]:
train['term'].dtype

In [ ]:
train['term'].value_counts()

In [ ]:
plot_feature(train, feature='term', name='Term', dtype='cat', keep_outliers=True)

Loan with longer terms seem to have a far higher probability of defaulting - hardly surprising

### 4.3.13 Loan title

This data is already included in purpose, hence I will drop it prior to modeling

In [ ]:
train['title'].dtype

In [ ]:
train['title'].nunique()

In [ ]:
train['title'].value_counts()[:10]

## 4.4 Feature engineering

I create a feature for debt-to-income - a common metric for credit applications. It was originally in the payments data, but I assumed it to be real-time and thus couldn't be used.

### 4.4.1 Debt-to-income

In [28]:
def feature_engineering(df):
    """
    Function to calcualte debt-to-income
    """
    
    df['dti'] = df['installment'] / (df['annual_inc'] / 12)

In [29]:
feature_engineering(train)

In [ ]:
plot_feature(train, feature='dti', name='Debt-to-income', dtype='num', keep_outliers=True)

Defaulters seem to have a higher dti 

# 5. Data pre-processing

In [30]:
def convert_date_to_num(date_col):
    """
    Function to convert date variables to number format
    """ 
    
    return date_col.apply(lambda x:(parse(x)-datetime(1900,1,1)).days)

In [31]:
def data_preprocessing(df, ohe=False):
    """
    Function for data pre-processing
    
    The parameter ohe lets the user choose whether to do one-hot-encoding or transform those variables to categoricals
    
    returns processed DataFrame 
    """
    df_new = df.copy()
    
    feature_engineering(df_new)
    
    # Columns to drop
    cols_to_drop = ['emp_title', 'zip_code', 'application_type', 'desc', 'funded_amnt', 'funded_amnt_inv', 'grade', 
                    'pymnt_plan', 'title', 'issue_d',]
    
    # Drop columns
    df_new.drop(labels=cols_to_drop, axis=1, inplace=True)
    
    # Transform date column to int
    df_new['earliest_cr_line'] = convert_date_to_num(df_new['earliest_cr_line'])
    
    # Clean employment length feature
    df_new['emp_length'].replace('10+ years', '10 years', inplace=True)
    df_new['emp_length'].replace('< 1 year', '0 years', inplace=True)
    df_new['emp_length'].replace('n/a', np.nan, inplace=True)
    df_new['emp_length'] = df_new['emp_length'].apply(lambda x: x if pd.isnull(x) else np.int8(x.split()[0]))
    
    # Clean home ownership feature
    df_new['home_ownership'].replace(to_replace=['NONE', 'ANY'], value='OTHER', inplace=True)
    
    cat_cols = df_new.select_dtypes(include=['object']).columns
    
    # Performns ohe or transforming to categoricals
    if ohe:
        dummies = pd.get_dummies(df_new[cat_cols])
        df_new = df_new.drop(cat_cols, axis=1).join(dummies)
    else:  
        for col in cat_cols:
            df_new[col] = df_new[col].astype('category')
        
    return df_new

In [32]:
# This DataFrame includes ohe vars and will be used for models that need ohe 
X_train_processed = data_preprocessing(X_train, ohe=True).reset_index(drop=True)

In [33]:
# This DataFrame has categorical vars as categorical dtype and will be used for models that support that
X_train_processed2 = data_preprocessing(X_train, ohe=False).reset_index(drop=True)

In [40]:
X_train_processed2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 202376 entries, 0 to 202375
Data columns (total 16 columns):
emp_length          194476 non-null float64
home_ownership      202376 non-null category
addr_state          202376 non-null category
annual_inc          202376 non-null float64
open_acc            202376 non-null float64
pub_rec             202376 non-null float64
total_acc           202376 non-null float64
delinq_2yrs         202376 non-null float64
earliest_cr_line    202376 non-null int64
term                202376 non-null category
sub_grade           202376 non-null category
purpose             202376 non-null category
loan_amnt           202376 non-null int64
int_rate            202376 non-null float64
installment         202376 non-null float64
dti                 202376 non-null float64
dtypes: category(5), float64(9), int64(2)
memory usage: 18.0 MB


In [38]:
print('X_train_processed (with ohe) shape is', X_train_processed.shape)
print('X_train_processed2 (with categoricals) shape is', X_train_processed2.shape)

X_train_processed (with ohe) shape is (202376, 117)
X_train_processed2 (with categoricals) shape is (202376, 16)


# 6. Modeling

The modeling process is done as following: <br>
1) Spot-check 3 different algos using GridSearchCV <br>
2) Optimize hyperparameters for the best performing algo using HypOpt on a sample of the training data <br>
3) See results on best model on full training data (still using CV) <br>
4) Look at feature importances <br>

## 6.1 Spot-check 3 different algorithms using grid search cross-validation

### 6.1.1 Logistic regression using stochastic gradient descent

In [ ]:
# Create pipeline with both imputation and standard scaling
logreg_pipeline = Pipeline([
    ('imputer', Imputer(strategy='median')),
    ('scaler', StandardScaler()),
    ('model', SGDClassifier(loss='log', max_iter=1000, tol=1e-3, random_state=seed, warm_start=True))
])

In [ ]:
# Create parameters grid
logreg_param_grid  = {
    'model__alpha': [10**-5, 10**-2, 10**1],
    'model__penalty': ['l1', 'l2']
}

In [ ]:
# Create GridSearchCV object for the logistic regression
GridSearchCV_logreg = GridSearchCV(estimator=logreg_pipeline, param_grid=logreg_param_grid,
                                   scoring='roc_auc', n_jobs=-1, pre_dispatch='2*n_jobs', cv=5, 
                                   verbose=1, return_train_score=True)

In [ ]:
# Fit to pre-processed training data (with ohe)
GridSearchCV_logreg.fit(X_train_processed, y_train)

In [ ]:
# The best roc-auc score amongst the iterations
GridSearchCV_logreg.best_score_

In [ ]:
# Best parameters
GridSearchCV_logreg.best_params_

### 6.1.2 Random Forest Classifier

In [ ]:
# Create pipeline for Random Forest Classifier. Scaling is not needed for tree-based models
rfc_pipeline = Pipeline([
    ('imputer', Imputer(strategy='median')),
    ('model', RandomForestClassifier(n_jobs=-1, random_state=seed))
])

I have to limit the RF grid because it's very computationally expensive

In [ ]:
rfc_param_grid  = {
    'model__n_estimators': [100],
    #'model__max_features': ['auto'],
    #'model__max_depth': [None, 3]
}
pprint(rfc_param_grid)

In [ ]:
# Create GridSearchCV object for RF
GridSearchCV_rfc = GridSearchCV(estimator=rfc_pipeline, param_grid=rfc_param_grid,
                                scoring='roc_auc', n_jobs=-1, pre_dispatch='2*n_jobs', cv=5, verbose=1)

In [ ]:
# Fit to pre-processed training data (with ohe)
GridSearchCV_rfc.fit(X_train_processed, y_train)

In [ ]:
GridSearchCV_rfc.best_score_

In [ ]:
GridSearchCV_rfc.best_params_

### 6.1.3 Gradient Boosting Classifier: LightGBM

I will use the LightGBM algo for Gradient Boosting, both due to its performance and speed. The LightGBM does not need ohe categorical variables, thus we will use the X_train_preprocessed2 dataframe for training. 

In [34]:
lgb_model = lgb.LGBMClassifier(seed = seed, boosting_type='gbdt')

In [35]:
# Create parameter grid for LGBM
lgb_param_grid  = {
    'learning_rate': [0.001, 0.01, 0.1],
    'subsample': [0.8, 1.0]
}

In [36]:
# Create GridSearchVB object for LGBM
GridSearchCV_lgb = GridSearchCV(estimator=lgb_model, param_grid=lgb_param_grid,
                                   scoring='roc_auc', n_jobs=-1, pre_dispatch='2*n_jobs', cv=5, 
                                   verbose=1, return_train_score=True)

In [37]:
GridSearchCV_lgb.fit(X_train_processed2, y_train)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:   40.3s finished


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        importance_type='split', learning_rate=0.1, max_depth=-1,
        min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
        n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
        random_state=None, reg_alpha=0.0, reg_lambda=0.0, seed=42,
        silent=True, subsample=1.0, subsample_for_bin=200000,
        subsample_freq=0),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'learning_rate': [0.001, 0.01, 0.1], 'subsample': [0.8, 1.0]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='roc_auc', verbose=1)

In [ ]:
GridSearchCV_lgb.best_score_

In [ ]:
GridSearchCV_lgb.best_params_

### 6.1.4 Compare algorithms in plot

To compare the algorithms in a plot, I will extract the scores from each cv with the best hyperparameters in the grid search

In [ ]:
# Use regex to match the keys of interst in the GridSearchCV object
regex = re.compile('split._test_score')

In [ ]:
models = [('SGD Logistic Regression', GridSearchCV_logreg),
            ('Random Forest Classifier', GridSearchCV_rfc),
            ('LightGBM', GridSearchCV_lgb)]

In [ ]:
# Create empty DataFrame for results
results = pd.DataFrame()

In [ ]:
# Iterate through each (name, GridSearchCV object) tuple in in the models list 
# and extract the results in the cv with the best hyperparameters
for name, grid_cv in models:
    ix = grid_cv.best_index_
    for key in grid_cv.cv_results_:
        if re.match(regex, key):
            results.loc[key[5], name] = grid_cv.cv_results_[key][ix]

In [ ]:
# Unstack the dataset for plotting
results = results.unstack().reset_index(level=0)
results.columns = ['Model', 'cv results']

In [ ]:
# Plot the results
fig, ax = plt.subplots(figsize=(8,3))
sns.swarmplot(x='Model', y='cv results', data=results, ax=ax)
ax.set_ylabel('Area under the AUC curve')

We see that the LightGBM Gradient Boosting algorithm performs consistently better, and always > 0.70 which is descent.

I will progress with the LightGBM model in the modeling process.

## 6.2 Bayesian Hyperparameter Optimization

I will use Bayesian Hyperparameter Optimization with the Hyperopt library to find optimal hyperparameters for the LightGBM model. Due to its computational requirement, I will sample the training dataset. As this is merely on the prototyping stage, this seems reasonable. At a later stage, one might want to search further with more iterations and the entire dataset.

### 6.2.1 Create sample dataset to limit computation time

In [ ]:
# Settings
max_evals = 1000
n_folds = 5

In [ ]:
# Concatenate X and y train datasets and sample 20k rows
train_bayes = pd.concat([y_train, X_train_processed2], axis=1)
train_bayes = train_bayes.sample(n= 20000, random_state=seed).reset_index(drop=True)

In [ ]:
# Split training dataset for hyperparameter optimization in X and y
y_train_bayes = np.array(train_bayes['target'])
X_train_bayes = train_bayes.drop('target', axis=1)
print('Train features shape: ', X_train_bayes.shape)
print('Train labels shape: ', y_train_bayes.shape)

In [ ]:
# Define dataset required for LGB cv method
train_set = lgb.Dataset(X_train_bayes, label = y_train_bayes)

### 6.2.2 Set up components: objective fuction, domain and optimization algorithm

Bayesian hyperparameter optimization requires 3 components: <br>
1) The objective function to optimize - in our situation the CV auc score for the LightGBM model <br>
2) Bayesian domain - equivalent to the hyperparameter grid, but one needs to define probability distributions to sample from <br>
3) Optmization algorithm - I will use the TPE (Tree Parzen Estimator)

In [ ]:
def objective(hyperparameters, seed=seed, n_folds=n_folds):
    """
    Objective function for Gradient Boosting Machine Hyperparameter Optimization.
    Writes a new line to 'outfile' on every iteration
    """
    
    # Keep track of evals
    global ITERATION
    
    ITERATION += 1
    
    # Using early stopping to find number of trees trained
    if 'n_estimators' in hyperparameters:
        del hyperparameters['n_estimators']
    
    # Make sure parameters that need to be integers are integers
    for parameter_name in ['num_leaves', 'subsample_for_bin', 'min_child_samples']:
        hyperparameters[parameter_name] = int(hyperparameters[parameter_name])

    start = timer()
    
    # Perform n_folds cross validation
    cv_results = lgb.cv(hyperparameters, train_set, num_boost_round = 10000, nfold = n_folds, 
                        early_stopping_rounds = 100, metrics = 'auc', seed = seed)

    run_time = timer() - start
    
    # Extract the best score
    best_score = cv_results['auc-mean'][-1]
    
    # Loss must be minimized
    loss = 1 - best_score
    
    # Boosting rounds that returned the highest cv score
    n_estimators = len(cv_results['auc-mean'])
    
    # Add the number of estimators to the hyperparameters
    hyperparameters['n_estimators'] = n_estimators

    # Write to the csv file ('a' means append)
    of_connection = open(OUT_FILE, 'a')
    writer = csv.writer(of_connection)
    writer.writerow([loss, hyperparameters, ITERATION, run_time, best_score])
    of_connection.close()

    # Dictionary with information for evaluation
    return {'loss': loss, 'hyperparameters': hyperparameters, 'iteration': ITERATION,
            'train_time': run_time, 'status': STATUS_OK}

In [ ]:
# Define the Bayesian Domain
space = {
    'boosting_type': 'gbdt',
    'subsample': hp.uniform('gdbt_subsample', 0.5, 1), 
    'num_leaves': hp.quniform('num_leaves', 20, 150, 1),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.01), np.log(0.5)),
    'subsample_for_bin': hp.quniform('subsample_for_bin', 20000, 300000, 20000),
    'min_child_samples': hp.quniform('min_child_samples', 20, 500, 5),
    'reg_alpha': hp.uniform('reg_alpha', 0.0, 1.0),
    'reg_lambda': hp.uniform('reg_lambda', 0.0, 1.0),
    'colsample_bytree': hp.uniform('colsample_by_tree', 0.6, 1.0),
    'is_unbalance': hp.choice('is_unbalance', [True, False]),
}

In [ ]:
# Create the optimization algorithm
tpe_algorithm = tpe.suggest

In [ ]:
# Record results
trials = Trials()

In [ ]:
# Create a file and open a connection
OUT_FILE = 'bayesian_hypopt_test.csv'
of_connection = open(OUT_FILE, 'w')
writer = csv.writer(of_connection)

ITERATION = 0

# Write column names
headers = ['loss', 'hyperparameters', 'iteration', 'runtime', 'score']
writer.writerow(headers)
of_connection.close()

### 6.2.3 Run the hyperparameter optimization

In [ ]:
# Global variable
global ITERATION

ITERATION = 0

# Run optimization
best = fmin(fn = objective, space = space, algo = tpe.suggest, trials = trials, max_evals = max_evals)

In [ ]:
# Print the best hyperparameters
best

In [ ]:
# Read in results recorded in the csv file 
results = pd.read_csv(OUT_FILE)

In [ ]:
#Print results sorted by score
results.sort_values(by='score', ascending=False)

### 6.2.4 Plot score results over iterations in the optimization process

In [ ]:
def evaluate(results, name):
    """Evaluate model on test data using hyperparameters in results
       Return dataframe of hyperparameters"""
    
    new_results = results.copy()
    # String to dictionary
    new_results['hyperparameters'] = new_results['hyperparameters'].map(ast.literal_eval)
    
    # Sort with best values on top
    new_results = new_results.sort_values('score', ascending = False).reset_index(drop = True)
    
    # Print out cross validation high score
    print('The highest cross validation score from {} was {:.5f} found on iteration {}.'.
          format(name, new_results.loc[0, 'score'], new_results.loc[0, 'iteration']))
    
    # Use best hyperparameters to create a model
    hyperparameters = new_results.loc[0, 'hyperparameters']
    model = lgb.LGBMClassifier(**hyperparameters)
    
    # Train and make predictions
    model.fit(X_train_bayes, y_train_bayes)
    
    # Create dataframe of hyperparameters
    hyp_df = pd.DataFrame(columns = list(new_results.loc[0, 'hyperparameters'].keys()))

    # Iterate through each set of hyperparameters that were evaluated
    for i, hyp in enumerate(new_results['hyperparameters']):
        hyp_df = hyp_df.append(pd.DataFrame(hyp, index = [0]), 
                               ignore_index = True)
        
    # Put the iteration and score in the hyperparameter dataframe
    hyp_df['iteration'] = new_results['iteration']
    hyp_df['score'] = new_results['score']
    
    return hyp_df

In [ ]:
bayes_params = evaluate(results, name = 'Bayesian')

In [ ]:
# Dataframe of just scores
scores = pd.DataFrame({'ROC AUC': bayes_params['score'], 'iteration': bayes_params['iteration']})

scores['ROC AUC'] = scores['ROC AUC'].astype(np.float32)
scores['iteration'] = scores['iteration'].astype(np.int32)

best_bayes_params = bayes_params.iloc[bayes_params['score'].idxmax(), :].copy()

In [ ]:
# Plot of scores over the course of searching
sns.lmplot('iteration', 'ROC AUC', data = scores, size = 3.5, aspect=2)
plt.scatter(best_bayes_params['iteration'], best_bayes_params['score'], marker = '*', s = 400, c = 'darkblue', edgecolor = 'k')
plt.xlabel('Iteration')
plt.ylabel('ROC AUC')
plt.title("Validation ROC AUC versus Iteration")

We note that there is a positive trend in the score outcome with respect to number of iterations - something we would expect for bayesian hyperparameter optimization

## 6.3 Results of best model on full train data

We have so far only shown results on a sample of the train data for the best model. It's time to test it on the full training data.

In [ ]:
# Define dataset required for LGB cv method
full_train_set = lgb.Dataset(X_train_processed2, label = y_train)

In [ ]:
# Cast variables to the appropriate data type
best['subsample_for_bin'] = int(best['subsample_for_bin'])
best['num_leaves'] = int(best['num_leaves'])
best['min_child_samples'] = int(best['min_child_samples'])
best['is_unbalance'] = bool(best['is_unbalance'])

In [ ]:
# Fit the model with optimal hyperparameters to the full training data
cv_result = lgb.cv(best, full_train_set, num_boost_round = 10000, nfold = n_folds, 
                        early_stopping_rounds = 100, metrics = 'auc', seed = seed)

In [ ]:
# Print number of boosting iterations used in the fitted model
len(cv_result['auc-mean'])

In [ ]:
# Print the auc score for the full dataset
cv_result['auc-mean'][-1]

I note that the roc-auc score with the optimal hyperparameters found in the bayesian optimization is actually lower than the spot-checked. The reason is very likely to be that we used sampled training data for the optimization. To enhance performance, one would need more computational power and run the hyperparameter optimization on the full dataset to begin with.

## 6.4 Feature importances with best model

In [ ]:
# Create variables for importance and labels and indeces to sort them in the next step
importances = best_lgb.feature_importances_
feat_labels = X_train_processed2.columns
indices = np.argsort(importances)[::-1]

In [ ]:
# plot the top_n features
fig, ax = plt.subplots(figsize=(10,6))    
top_n = 10
ax.set_title('Feature Importances')
ax.barh(range(top_n), importances[indices[0:top_n]], align = 'center')
plt.yticks(range(top_n), feat_labels[indices[0:top_n]], rotation=0)
plt.ylim([-1, top_n])
plt.show()

# 7. Results on Test Data

## 7.1 Pre-process test data

I will use the same data pre-processing function that we used for the train data. One-hot-encoding is not needed as the LightGBM handles categorical variables.

In [ ]:
X_test.shape

In [ ]:
X_test_processed = data_preprocessing(X_test, ohe=False)

In [ ]:
X_test_processed.shape

## 7.2 Confusion matrix

In [ ]:
def adjusted_classes(y_scores, t):
    """
    This function adjusts class predictions based on the prediction threshold (t).
    """
    
    return [1 if y >= t else 0 for y in y_scores]

In [ ]:
def cm_by_threshold(y_scores, y_actual, t):
    """
    Returns the confusion matrix by threshold
    """

    # Run function to adjust class prediction by threshold
    pred_adj = adjusted_classes(y_scores, t)
    
    return confusion_matrix(y_actual, pred_adj)

In [ ]:
def plot_confusion_matrix(y_scores, y_actual, classes,
                          t, normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    
    cm = cm_by_threshold(y_scores, y_actual, t=t)
    
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    plt.figure(figsize=(4,3))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

In [ ]:
# Extract prediction scores for the test data
y_test_scores = best_lgb.predict_proba(X_test_processed)[:,1]

In [ ]:
# Plot confusion matrix
plot_confusion_matrix(y_test_scores, y_test, ['Non-default', 'Default'], t=0.70, normalize=False)

## 7.3 ROC curve and ROC AUC score

In [ ]:
def plot_roc_curve(fpr, tpr, roc_auc):
    """
    Function to plot the roc curve
    """
    
    plt.figure(figsize=(6,4))
    plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % (roc_auc))
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.0])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic')
    plt.legend(loc="lower right")

In [ ]:
# Calculate the roc_auc for the test data
roc_auc = roc_auc_score(y_test, y_test_scores)

In [ ]:
print('The Area Under the ROC curve is', roc_auc)

One can note that the auc is actually higher for the test data than for the training data!

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, y_test_scores)

In [ ]:
plot_roc_curve(fpr, tpr, roc_auc)

The result on the testing data looks very promising, with a descent shaped ROC curve.

One would need to carefully select the threshold based on the preference on TPR vs FPR.

## 7.4 Comparison with sub-grade model

Let's compare the model's results with one that might be in place today - based on the subgrades. By mapping a probability linearly along the subgrades (giving 0% pd on A1 and 50% pd on G5), we might get something that is similar to what is being used today. By building a confusion matrix and retrieving the roc-auc for the sub-grade model, we can judge how much value the machine learning model adds.

In [ ]:
# Create sorted list of subgrades
sorted_subgrades = sorted(X_test['sub_grade'].unique())

In [ ]:
# Create a DataFrame for subgrades with probability of default as a linear function from 0 to 50%
subgrades_df = pd.DataFrame({'sub_grade': sorted_subgrades, 
                             'pd': np.linspace(0, 0.5, len(sorted_subgrades))}).set_index('sub_grade', drop=True)

In [ ]:
# Map in default probabilities in test dataset
y_test_scores_subgrades = X_test['sub_grade'].map(lambda x: subgrades_df.loc[x][0])

In [ ]:
# Plot confusion matrix with a threshold that returns a similar number of TP as our ML model
plot_confusion_matrix(y_test_scores_subgrades, y_test, ['Non-default', 'Default'], t=0.17, normalize=False)

By tweeking the threshold of the subgrade model to match the ML model's number of TPs, one can compare the performance by looking at the confusion matrices. The ML model clearly outperforms the subgrade model with 201 less FNs and 1307 less FPs, clearly demonstrating the ML model’s added business value.

In [ ]:
# Retrieve roc_auc score for the sub-grade model
roc_auc_subgrades = roc_auc_score(y_test, y_test_scores_subgrades)

In [ ]:
print('The Area Under the ROC curve is', roc_auc_subgrades)

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, y_test_scores_subgrades)

In [ ]:
plot_roc_curve(fpr, tpr, roc_auc_subgrades)